In [3]:
import pandas as pd
import numpy as np
import re


In [6]:
# Reload the uploaded Excel file after environment reset
file_path = "Diversity-Inclusion-Dataset.xlsx"
df = pd.read_excel(file_path, sheet_name="Pharma Group AG")

In [7]:
# Helper: standardize Yes/No to boolean
def yn_to_bool(val):
    if pd.isna(val):
        return np.nan
    s = str(val).strip().lower()
    if s in {"yes", "y", "true", "t", "1"}:
        return True
    if s in {"no", "n", "false", "f", "0"}:
        return False
    return np.nan

In [8]:
# Helper: split "6 - Junior Officer" -> (6, "Junior Officer")
def split_job_level(s):
    if pd.isna(s):
        return (np.nan, np.nan)
    m = re.match(r"\s*(\d+)\s*-\s*(.+)$", str(s).strip())
    if m:
        return (int(m.group(1)), m.group(2).strip())
    try:
        n = int(str(s).strip())
        return (n, np.nan)
    except:
        return (np.nan, str(s).strip())

In [9]:
# Clean relevant columns
df_clean = pd.DataFrame({
    "employee_id": df["Employee ID"],
    "gender": df["Gender"].astype(str).str.strip().str.title(),
    "department": df["Department @01.07.2020"].astype(str).str.strip().str.title(),
    "job_level_before_raw": df["Job Level before FY20 promotions"],
    "job_level_after_raw": df["Job Level after FY20 promotions"],
    "new_hire_fy20": df["New hire FY20?"].apply(yn_to_bool),
    "fy20_perf_rating": pd.to_numeric(df["FY20 Performance Rating"], errors="coerce"),
    "fy19_perf_rating": pd.to_numeric(df["FY19 Performance Rating"], errors="coerce"),
    "promotion_in_fy20": df["Promotion in FY20?"].apply(yn_to_bool),
    "promotion_in_fy21": df["Promotion in FY21?"].apply(yn_to_bool),
    "in_base_group_for_promo_fy21": df["In base group for Promotion FY21"].apply(yn_to_bool),
    "fy20_leaver": df["FY20 leaver?"].apply(yn_to_bool),
    "leaver_fy": df["Leaver FY"],
    "in_base_group_for_turnover_fy20": df["In base group for turnover FY20"].apply(yn_to_bool),
    "nationality": df["Nationality 1"].astype(str).str.strip().str.title(),
    "region_group": df["Region group: nationality 1"].astype(str).str.strip().str.title(),
    "broad_region_group": df["Broad region group: nationality 1"].astype(str).str.strip().str.title(),
    "age_group": df["Age group"],
    "last_hire_date": pd.to_datetime(df["Last hire date"], errors="coerce"),
    "time_in_job_level_at_2020_07_01": df["Time in Job Level @01.07.2020"]
})

In [10]:
# Parse job level fields
before_split = df_clean["job_level_before_raw"].apply(split_job_level)
after_split = df_clean["job_level_after_raw"].apply(split_job_level)

In [11]:
df_clean["job_level_before_num"] = before_split.apply(lambda x: x[0])
df_clean["job_level_before_title"] = before_split.apply(lambda x: x[1])
df_clean["job_level_after_num"] = after_split.apply(lambda x: x[0])
df_clean["job_level_after_title"] = after_split.apply(lambda x: x[1])

In [12]:
# Derive is_active_fy20
df_clean["is_active_fy20"] = df_clean["fy20_leaver"].map(lambda x: False if x is True else True)

In [13]:
# Seniority group
def seniority_group(n):
    try:
        if np.isnan(n):
            return np.nan
        n = int(n)
        if n <= 2:
            return "Senior Leadership"
        elif n <= 4:
            return "Mid Management"
        else:
            return "Junior/IC"
    except:
        return np.nan

In [14]:
df_clean["seniority_group_after"] = df_clean["job_level_after_num"].apply(seniority_group)

In [15]:
# Reorder useful columns
ordered_cols = [
    "employee_id", "gender", "age_group", "nationality", "region_group", "broad_region_group",
    "department",
    "job_level_before_num", "job_level_before_title",
    "job_level_after_num", "job_level_after_title", "seniority_group_after",
    "fy19_perf_rating", "fy20_perf_rating",
    "new_hire_fy20", "fy20_leaver", "is_active_fy20", "leaver_fy",
    "promotion_in_fy20", "promotion_in_fy21", "in_base_group_for_promo_fy21",
    "in_base_group_for_turnover_fy20",
    "last_hire_date", "time_in_job_level_at_2020_07_01"
]

In [16]:
df_clean = df_clean[ordered_cols]

In [18]:
# Save cleaned data
out_csv = "id_hr_clean.csv"
df_clean.to_csv(out_csv, index=False)

In [23]:
# Prepare SQL DDL
out_sql = "id_hr_ddl.sql"
ddl = """
CREATE TABLE hr_clean (
  employee_id INT PRIMARY KEY,
  gender TEXT,
  age_group TEXT,
  nationality TEXT,
  region_group TEXT,
  broad_region_group TEXT,
  department TEXT,
  job_level_before_num INT,
  job_level_before_title TEXT,
  job_level_after_num INT,
  job_level_after_title TEXT,
  seniority_group_after TEXT,
  fy19_perf_rating NUMERIC,
  fy20_perf_rating NUMERIC,
  new_hire_fy20 BOOLEAN,
  fy20_leaver BOOLEAN,
  is_active_fy20 BOOLEAN,
  leaver_fy TEXT,
  promotion_in_fy20 BOOLEAN,
  promotion_in_fy21 BOOLEAN,
  in_base_group_for_promo_fy21 BOOLEAN,
  in_base_group_for_turnover_fy20 BOOLEAN,
  last_hire_date DATE,
  time_in_job_level_at_2020_07_01 TEXT
);
"""

In [24]:
with open(out_sql, "w") as f:
    f.write(ddl)

In [25]:
# Show preview
df_clean.head(10)

,employee_id,gender,age_group,nationality,region_group,broad_region_group,department,job_level_before_num,job_level_before_title,job_level_after_num,...,new_hire_fy20,fy20_leaver,is_active_fy20,leaver_fy,promotion_in_fy20,promotion_in_fy21,in_base_group_for_promo_fy21,in_base_group_for_turnover_fy20,last_hire_date,time_in_job_level_at_2020_07_01
0,1,Male,30 to 39,Spain,Europe,Europe,Operations,6.0,Junior Officer,6,...,False,True,False,FY20,False,False,False,True,2017-04-01,3
1,2,Female,30 to 39,Germany,Europe,Europe,Sales & Marketing,4.0,Manager,4,...,False,False,True,NaN,False,False,True,True,2017-04-01,3
2,3,Male,30 to 39,Switzerland,Switzerland,Switzerland,Strategy,2.0,Director,2,...,False,False,True,NaN,False,False,True,True,2015-04-01,3
3,4,Male,30 to 39,Germany,Europe,Europe,Hr,4.0,Manager,4,...,False,False,True,NaN,False,False,True,True,2012-04-01,3
4,5,Female,20 to 29,Switzerland,Switzerland,Switzerland,Sales & Marketing,6.0,Junior Officer,6,...,False,False,True,NaN,False,False,True,True,2019-04-01,1
5,6,Female,40 to 49,Italy,Europe,Europe,Internal Services,NaN,NaN,4,...,True,False,True,NaN,False,False,False,False,2020-04-01,0
6,7,Male,30 to 39,Italy,Europe,Europe,Operations,NaN,NaN,3,...,True,False,True,NaN,False,False,False,False,2020-04-01,0
7,8,Female,30 to 39,Switzerland,Switzerland,Switzerland,Hr,5.0,Senior Officer,5,...,False,False,True,NaN,False,False,True,True,2013-04-01,3
8,9,Female,20 to 29,United States,Americas,Elsewhere,Sales & Marketing,6.0,Junior Officer,6,...,False,False,True,NaN,False,False,True,True,2019-04-01,1
9,10,Male,20 to 29,Switzerland,Switzerland,Switzerland,Internal Services,6.0,Junior Officer,6,...,False,False,True,NaN,False,False,True,True,2018-04-01,2
